# Artwork Puzzle Solution
This is my solution to the ["Artwork" puzzle](https://open.kattis.com/problems/artwork) on [open.kattis.com](open.kattis.com).

## Approach: Euler Characteristic
The key idea to this solution is to find the [Euler characteristic](https://en.wikipedia.org/wiki/Euler_characteristic), or equivalently the [rank](https://en.wikipedia.org/wiki/Rank_(graph_theory)), of the [dual graph](https://en.wikipedia.org/wiki/Dual_graph) to the painting.

The Euler characteristic for a graph with multiple connected components is
$$ \text{Vertices} - \text{Edges} + \text{Faces} - \text{Components} = 1$$
For a planar graph, this quantity is always equal to 1, so to find the beauty of the painting (the number of connected components), we simply need to calculate.
$$ \text{beauty} = V - E + F - 1 $$
The tricky part is finding a graph that corresponds to a given painting where we can efficiently determine $V$, $E$, and $F$. But, we're helped by a few properties of the prompt:
1. We always start with the $m \times n$ square grid.
2. We are finding the beauty after each (straight line) paint stroke.
3. We are only adding paint strokes. That is, once a square is painted, it will never be unpainted.

These together mean that:
1. We're working with a relatively simple graph that's guaranteed to be planar.
2. Each new graph (after adding each stroke) is a subgraph of the previous one.
3. We're always starting with a beauty of 1.
4. We only care about the *change* in beauty.

In [15]:
# Initialize some stuff first so it doesn't count against my time.
from numpy.random import *
import numpy as np
import time

seed(10989)
m = 1000
n = 1000
q = 10000

def randomstroke(m,n):
    x1=randint(1,m+1)
    y1=randint(1,n+1)
    coinflip=randint(2)
    if coinflip:
        x2=randint(1,m+1)
        y2=y1
    else:
        x2=x1
        y2=randint(1,n+1)
    return '{0} {1} {2} {3}'.format(x1,y1,x2,y2)
#print(randomstroke(m,n))

def makestrokelist(m,n,q):
    slist = []
    for nq in range(q):
        slist.append(randomstroke(m,n))
    return slist

strokelist = makestrokelist(m,n,q)

# Actual solution starts here.
start = time.time()

beauty = 1
    
squares = np.ones((m,n,5),dtype=bool)
squares[0,:,1]=0
squares[m-1,:,2]=0
squares[:,0,3]=0
squares[:,n-1,4]=0
faceref = list(range(0,m*(n-1)))
for i in range(m,len(faceref),m):
    faceref[i]=0

def faces(x,y):
    '''
    x and y are 1-indexed as in the prompt.
    c1, c2, c3, c4 are upper-left, upper-right, lower-left, lower-right.
    '''
    c1 = 0 if (x==1 or y==1) else m*(y-2)+x-1
    c2 = 0 if (x==m or y==1) else m*(y-2)+x
    c3 = 0 if (x==1 or y==n) else m*(y-1)+x-1
    c4 = 0 if (x==m or y==n) else m*(y-1)+x
    return faceref[c1],faceref[c2],faceref[c3],faceref[c4]

def paintstroke(stroke):    
    global beauty
    mergeset=set()
    
    def addtomerge(face):
        while face != faceref[face]:
            face = faceref[face]
        mergeset.add(face)

    def paintsquare(x,y):
        global beauty
        if squares[x-1,y-1,0]:
            beauty-=1
            squares[x-1,y-1,0]=0
            f1,f2,f3,f4 = faces(x,y)
            if squares[x-1,y-1,1]:
                beauty+=1
                squares[x-1,y-1,1]=0
                addtomerge(f1)
                addtomerge(f3)
                if x>1:
                    squares[x-2,y-1,2]=0
            if squares[x-1,y-1,2]:
                beauty+=1
                squares[x-1,y-1,2]=0
                addtomerge(f2)
                addtomerge(f4)
                if x<m:
                    squares[x,y-1,1]=0
            if squares[x-1,y-1,3]:
                beauty+=1
                squares[x-1,y-1,3]=0
                addtomerge(f1)
                addtomerge(f2)
                if y>1:
                    squares[x-1,y-2,4]=0
            if squares[x-1,y-1,4]:
                beauty+=1
                squares[x-1,y-1,4]=0
                addtomerge(f3)
                addtomerge(f4)
                if y<n:
                    squares[x-1,y,3]=0
        
    x1,y1,x2,y2 = (int(n) for n in stroke.split())
    for xi in range(x1,x2+1):
        for yi in range(y1,y2+1):
            paintsquare(xi,yi)
    #print(mergeset)
    if mergeset:
        beauty-=(len(mergeset)-1)
        minf = min(mergeset)
        while minf!=faceref[minf]:
            print("faceref needed to be fixed.")
            minf = faceref[minf]
        for s in mergeset:
            faceref[s]=minf
        
for qi in range(q):
    paintstroke(strokelist[qi])
    print(beauty)
    
end = time.time()

print(end-start)

9.891249895095825
